# 项目简介
该项目使用PaddleX快速训练垃圾分类模型，然后通过PaddleLite部署到华为Mate20 Pro手机上，实现飞桨框架深度学习模型的落地。

- 模型训练：PaddleX，`YoloV3`的`backbone`使用`MobileNetV3_large`和`DarkNet53`
- 模型转换：Paddle-Lite
- Android开发环境：Android Studio on Ubuntu 18.04 64-bit
- 移动端设备：华为Mate20 Pro

## 关于本项目
> 针对项目还存在的改进空间，以及其它模型在不同移动设备的部署，希望大家多交流观点、介绍经验，共同学习进步。[个人主页](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/90149)
# PaddleX快速训练迁移学习模型
## PaddleX简介
> [PaddleX](https://gitee.com/paddlepaddle/PaddleX)是基于飞桨核心框架、开发套件和工具组件的深度学习全流程开发工具。具备全流程打通、融合产业实践、易用易集成三大特点。
> 
> [PaddleX文档](https://paddlex.readthedocs.io/)
## 安装工具库

In [9]:
!pip install paddlelite
!pip install paddlex
!pip install shapely

## 准备数据集
本项目为目标检测场景，使用的垃圾分类数据集标注为COCO格式，包含图像文件夹及图像标注信息文件。
参考数据文件结构如下：
```
./dataset/  # 数据集根目录
|--JPEGImages  # 图像目录
|  |--xxx1.jpg
|  |--...
|  └--...
|
|--train.json  # 训练相关信息文件
|
└--val.json  # 验证相关信息文件

```
其中，相应的文件名可根据需要自行定义。

`train.json`和`val.json`存储与标注信息、图像文件相关的信息。如下所示：

```
{
  "annotations": [
    {
      "iscrowd": 0,
      "category_id": 1,
      "id": 1,
      "area": 33672.0,
      "image_id": 1,
      "bbox": [232, 32, 138, 244],
      "segmentation": [[32, 168, 365, 117, ...]]
    },
    ...
  ],
  "images": [
    {
      "file_name": "xxx1.jpg",
      "height": 512,
      "id": 267,
      "width": 612
    },
    ...
  ]
  "categories": [
    {
      "name": "labelA",
      "id": 1,
      "supercategory": "component"
    }
  ]
}
```
其中，每个字段的含义如下所示：

| 域名 | 字段名 | 含义 | 数据类型 | 备注 |
|:-----|:--------|:------------|------|:-----|
| annotations | id | 标注信息id | int | 从1开始 |
| annotations | iscrowd      | 标注框是否为一组对象 | int | 只有0、1两种取值 |
| annotations | category_id  | 标注框类别id | int |  |
| annotations | area         | 标注框的面积 | float |  |
| annotations | image_id     | 当前标注信息所在图像的id | int |  |
| annotations | bbox         | 标注框坐标 | list | 长度为4，分别代表x,y,w,h |
| annotations | segmentation | 标注区域坐标 | list | list中有至少1个list，每个list由每个小区域坐标点的横纵坐标(x,y)组成 |
| images          | id                | 图像id | int | 从1开始 |
| images   | file_name         | 图像文件名 | str |  |
| images      | height            | 图像高度 | int |  |
| images       | width             | 图像宽度 | int |  |
| categories  | id            | 类别id | int | 从1开始 |
| categories | name          | 类别标签名 | str |  |
| categories | supercategory | 类别父类的标签名 | str |  |

其他格式数据准备方式请参考官方文档：
[数据集格式说明](https://gitee.com/paddlepaddle/PaddleX/blob/develop/docs/appendix/datasets.md)

In [ ]:
! wget https://bj.bcebos.com/paddlex/datasets/garbage_ins_det.tar.gz
! tar xzf garbage_ins_det.tar.gz

--2020-07-05 11:12:45--  https://bj.bcebos.com/paddlex/datasets/garbage_ins_det.tar.gz
Resolving bj.bcebos.com (bj.bcebos.com)... 182.61.200.229, 182.61.200.195
Connecting to bj.bcebos.com (bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196078673 (187M) [application/octet-stream]
Saving to: ‘garbage_ins_det.tar.gz’

garbage_ins_det.tar 100%[===================>] 187.00M  30.0MB/s    in 11s     

2020-07-05 11:12:56 (17.3 MB/s) - ‘garbage_ins_det.tar.gz’ saved [196078673/196078673]



## 模型训练
### 配置GPU环境

In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

### 数据处理与数据增强
更多详细的数据增强组合请参考[官方文档](https://gitee.com/paddlepaddle/PaddleX/blob/develop/docs/apis/transforms/det_transforms.md)

In [13]:
from paddlex.det import transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=512, max_size=512),
    transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=512, max_size=512),
    transforms.Padding(coarsest_stride=32),
])

### 创建数据读取器
关于其它格式目标检测数据集的读取方式请参考[检测和实例分割数据集](https://gitee.com/paddlepaddle/PaddleX/blob/develop/docs/apis/datasets/detection.md)

In [14]:
train_dataset = pdx.datasets.CocoDetection(
    data_dir='garbage_ins_det/JPEGImages',
    ann_file='garbage_ins_det/train.json',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='garbage_ins_det/JPEGImages',
    ann_file='garbage_ins_det/val.json',
    transforms=eval_transforms)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
2020-07-06 20:44:57 [INFO]	Starting to read file list from dataset...
2020-07-06 20:44:57 [INFO]	221 samples in file garbage_ins_det/train.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
2020-07-06 20:44:57 [INFO]	Starting to read file list from dataset...
2020-07-06 20:44:57 [INFO]	62 samples in file garbage_ins_det/val.json


### 使用`MobileNetV3_large`做`backbone`训练YoloV3迁移学习模型

In [1]:
num_classes = len(train_dataset.labels)
model = pdx.det.YOLOv3(num_classes=num_classes, backbone='MobileNetV3_large')
model.train(
    num_epochs=270,
    train_dataset=train_dataset,
    train_batch_size=8,
    eval_dataset=eval_dataset,
    learning_rate=0.000125,
    lr_decay_epochs=[210, 240],
    save_dir='output/yolov3_mobilevetv3',
    use_vdl=True)

#### 模型效果评估
![file](https://ai-studio-static-online.cdn.bcebos.com/911351e30dd44ca187a2edf60259f3ac341eb627dfcb4e979fc48f8c197ee59a)

In [1]:
eval_details_file = 'output/yolov3_mobilevetv3/epoch_270/eval_details.json'
pdx.det.draw_pr_curve(eval_details_file, save_dir='./garbage_ins_det')

In [16]:
model = pdx.load_model('output/yolov3_mobilevetv3/epoch_270')
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='garbage_ins_det/JPEGImages',
    ann_file='garbage_ins_det/test.json',
    transforms=eval_transforms)
metrics, evaluate_details = model.evaluate(eval_dataset, batch_size=8, return_details=True)
gt = evaluate_details['gt']
bbox = evaluate_details['bbox']
pdx.det.draw_pr_curve(gt=gt, pred_bbox=bbox, save_dir='./garbage_ins_det')

2020-07-06 20:45:52 [INFO]	Model[YOLOv3] loaded.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
2020-07-06 20:45:52 [INFO]	Starting to read file list from dataset...
2020-07-06 20:45:52 [INFO]	31 samples in file garbage_ins_det/test.json
2020-07-06 20:45:52 [INFO]	Start to evaluating(total_samples=31, total_steps=4)...


100%|██████████| 4/4 [00:10<00:00,  2.65s/it]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.06s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.286
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.741
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.192
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.288
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.243
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.390
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.390
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.

In [3]:
%mkdir log
%cp output/yolov3_mobilevetv3/vdl_log/*.log log/

查看VisualDL训练过程，需要执行如下步骤：
1. 将`output/yolov3_mobilevetv3/vdl_log`目录下的`.log`文件移动到`/home/aistudio/log`目录下
2. 打开终端执行命令`visualdl --logdir ./log --port 8001`
3. 复制本项目的网址并将`notebooks`之后内容全部替换为`visualdl`，如打开网页`https://aistudio.baidu.com/bdvgpu32g/user/90149/613622/visualdl`

![file](https://ai-studio-static-online.cdn.bcebos.com/1901479b637a4df9a7528a10a0f70e333d5daa081f134c0ba8acbe0a4842e6fd)

### 使用`DarkNet53`做`backbone`训练YoloV3迁移学习模型

In [2]:
num_classes = len(train_dataset.labels)
model = pdx.det.YOLOv3(num_classes=num_classes, backbone='DarkNet53')
model.train(
    num_epochs=270,
    train_dataset=train_dataset,
    train_batch_size=8,
    eval_dataset=eval_dataset,
    learning_rate=0.000125,
    lr_decay_epochs=[210, 240],
    save_dir='output/yolov3_darknet53',
    use_vdl=True)

#### 模型效果评估
![file](https://ai-studio-static-online.cdn.bcebos.com/484a0e622e1b4699b535f85e7b4a908b18cb457477bd4184afc05447572e5f67)

In [2]:
eval_details_file = 'output/yolov3_darknet53/epoch_270/eval_details.json'
pdx.det.draw_pr_curve(eval_details_file, save_dir='./garbage_ins_det')

In [18]:
model = pdx.load_model('output/yolov3_darknet53/epoch_270')
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='garbage_ins_det/JPEGImages',
    ann_file='garbage_ins_det/test.json',
    transforms=eval_transforms)
metrics, evaluate_details = model.evaluate(eval_dataset, batch_size=8, return_details=True)
gt = evaluate_details['gt']
bbox = evaluate_details['bbox']
pdx.det.draw_pr_curve(gt=gt, pred_bbox=bbox, save_dir='./garbage_ins_det')

2020-07-06 20:47:48 [INFO]	Model[YOLOv3] loaded.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
2020-07-06 20:47:48 [INFO]	Starting to read file list from dataset...
2020-07-06 20:47:48 [INFO]	31 samples in file garbage_ins_det/test.json
2020-07-06 20:47:48 [INFO]	Start to evaluating(total_samples=31, total_steps=4)...


100%|██████████| 4/4 [00:03<00:00,  1.32it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.773
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.312
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.245
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.403
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.403
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.

### 训练FasterRCNN模型（转换Paddle-Lite模型不成功，没有实际部署）


In [4]:
num_classes = len(train_dataset.labels) + 1
model = pdx.det.FasterRCNN(num_classes=num_classes)

In [5]:
model.train(
    num_epochs=12,
    train_dataset=train_dataset,
    train_batch_size=2,
    eval_dataset=eval_dataset,
    learning_rate=0.0025,
    lr_decay_epochs=[8, 11],
    save_dir='output/faster_rcnn_r50_fpn',
    use_vdl=True)

#### 模型效果评估
![file](https://ai-studio-static-online.cdn.bcebos.com/9e138e484eae4d9d9c155018f706c5d5d7f66e5e59b546c0a546776fb6ac0cc7)

In [3]:
eval_details_file = 'output/faster_rcnn_r50_fpn/epoch_12/eval_details.json'
pdx.det.draw_pr_curve(eval_details_file, save_dir='./garbage_ins_det')

In [20]:
model = pdx.load_model('output/faster_rcnn_r50_fpn/epoch_12')
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='garbage_ins_det/JPEGImages',
    ann_file='garbage_ins_det/test.json',
    transforms=eval_transforms)
metrics, evaluate_details = model.evaluate(eval_dataset, batch_size=8, return_details=True)
gt = evaluate_details['gt']
bbox = evaluate_details['bbox']
pdx.det.draw_pr_curve(gt=gt, pred_bbox=bbox, save_dir='./garbage_ins_det')

2020-07-06 20:49:07 [INFO]	Model[FasterRCNN] loaded.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
2020-07-06 20:49:07 [INFO]	Starting to read file list from dataset...
2020-07-06 20:49:07 [INFO]	31 samples in file garbage_ins_det/test.json
2020-07-06 20:49:07 [WARNING]	Faster RCNN supports batch_size=1 only during evaluating, so batch_size is forced to be set to 1.
2020-07-06 20:49:07 [INFO]	Start to evaluating(total_samples=31, total_steps=31)...


100%|██████████| 31/31 [00:04<00:00,  6.82it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.06s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.867
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.667
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.869
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.509
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.885
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.885
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.

# 模型部署准备
## Paddle-Lite简介
> [Paddle Lite](https://gitee.com/paddlepaddle/paddle-lite)为Paddle-Mobile的升级版，定位支持包括手机移动端在内更多场景的轻量化高效预测，支持更广泛的硬件和平台，是一个高性能、轻量级的深度学习预测引擎。在保持和PaddlePaddle无缝对接外，也兼容支持其他训练框架产出的模型。
> 
> 完整使用文档位于 [PaddleLite 文档](https://paddle-lite.readthedocs.io/zh/latest/) 。
## 导出inference模型

参考PaddleX文档：在服务端部署的模型需要首先将模型导出为inference格式模型，导出的模型将包括`__model__`、`__params__`和`model.yml`三个文名，分别为模型的网络结构，模型权重和模型的配置文件（包括数据预处理参数等等）。在安装完PaddleX后，在命令行终端使用如下命令导出模型到当前目录`inferece_model`下。

In [21]:
!paddlex --export_inference --model_dir=output/yolov3_mobilevetv3/epoch_270 --save_dir=./inference_model/yolov3_mobilevetv3

W0706 20:52:39.260469  1399 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 10.1, Runtime API Version: 9.0
W0706 20:52:39.264473  1399 device_context.cc:260] device: 0, cuDNN Version: 7.3.
2020-07-06 20:52:42 [INFO]	Model[YOLOv3] loaded.
2020-07-06 20:52:43 [INFO]	Model for inference deploy saved in ./inference_model/yolov3_mobilevetv3.


In [8]:
!paddlex --export_inference --model_dir=output/yolov3_darknet53/epoch_270 --save_dir=./inference_model/yolov3_darknet53

W0705 13:38:49.321007  1546 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 9.2, Runtime API Version: 9.0
W0705 13:38:49.325225  1546 device_context.cc:260] device: 0, cuDNN Version: 7.3.
2020-07-05 13:38:53 [INFO]	Model[YOLOv3] loaded.
2020-07-05 13:38:55 [INFO]	Model for inference deploy saved in ./inference_model/yolov3_darknet53.


In [27]:
!paddlex --export_inference --model_dir=output/faster_rcnn_r50_fpn/epoch_12 --save_dir=./inference_model/faster_rcnn_r50_fpn

W0706 21:16:32.794881  1451 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 10.1, Runtime API Version: 9.0
W0706 21:16:32.798995  1451 device_context.cc:260] device: 0, cuDNN Version: 7.3.
2020-07-06 21:16:36 [INFO]	Model[FasterRCNN] loaded.
2020-07-06 21:16:37 [INFO]	Model for inference deploy saved in ./inference_model/faster_rcnn_r50_fpn.


## 转换PaddleLite模型
这里没有使用PaddleX的deploy模块，会出现报错，具体原因待查。由于前面导出的模型是combined形式：模型文件夹model_dir下只有一个模型文件__model__和一个参数文件__params__；参考PaddleLite的文档，执行opt转换时只需要传入模型文件和参数文件路径。

官方文档给出的***详尽的转化命令***总结
```shell
paddle_lite_opt \
    --model_dir=<model_param_dir> \
    --model_file=<model_path> \
    --param_file=<param_path> \
    --optimize_out_type=(protobuf|naive_buffer) \
    --optimize_out=<output_optimize_model_dir> \
    --valid_targets=(arm|opencl|x86|npu|xpu) \
    --record_tailoring_info =(true|false)
```

| 选项         | 说明 |
| ------------------- | ------------------------------------------------------------ |
| --model_dir         | 待优化的PaddlePaddle模型（非combined形式）的路径 |
| --model_file        | 待优化的PaddlePaddle模型（combined形式）的网络结构文件路径。 |
| --param_file        | 待优化的PaddlePaddle模型（combined形式）的权重文件路径。 |
| --optimize_out_type | 输出模型类型，目前支持两种类型：protobuf和naive_buffer，其中naive_buffer是一种更轻量级的序列化/反序列化实现。若您需要在mobile端执行模型预测，请将此选项设置为naive_buffer。默认为protobuf。 |
| --optimize_out      | 优化模型的输出路径。                                         |
| --valid_targets     | 指定模型可执行的backend，默认为arm。目前可支持x86、arm、opencl、npu、xpu，可以同时指定多个backend(以空格分隔)，Model Optimize Tool将会自动选择最佳方式。如果需要支持华为NPU（Kirin 810/990 Soc搭载的达芬奇架构NPU），应当设置为npu, arm。 |
| --record_tailoring_info | 当使用 [根据模型裁剪库文件](./library_tailoring.html) 功能时，则设置该选项为true，以记录优化后模型含有的kernel和OP信息，默认为false。 |

* 如果待优化的fluid模型是非combined形式，请设置`--model_dir`，忽略`--model_file`和`--param_file`。
* 如果待优化的fluid模型是combined形式，请设置`--model_file`和`--param_file`，忽略`--model_dir`。
* 优化后的模型为以`.nb`名称结尾的单个文件。
* 删除`prefer_int8_kernel`的输入参数，`opt`自动判别是否是量化模型，进行相应的优化操作。

关于[python调用opt转化模型](https://gitee.com/paddlepaddle/paddle-lite/blob/develop/docs/user_guides/opt/opt_python.md)的详细参数设置介绍，可以参考官方文档。

### 统计模型算子信息、判断是否支持
> 注意：即使显示支持模型转换，实际也不一定能转换成功，比如该项目中的`faster_rcnn_r50_fpn`模型，具体原因待查

In [24]:
!paddle_lite_opt --print_model_ops=true  --model_dir=inference_model/yolov3_mobilevetv3 --valid_targets=arm

In [25]:
!paddle_lite_opt --print_model_ops=true  --model_dir=inference_model/yolov3_darknet53 --valid_targets=npu,arm

In [28]:
!paddle_lite_opt --print_model_ops=true  --model_dir=inference_model/faster_rcnn_r50_fpn --valid_targets=arm

OPs in the input model include:
                                        OP_name      Host       X86      CUDA       ARM    OpenCL      FPGA       NPU       XPU     RKNPU       APU       Any       Unk
                               anchor_generator                                       Y                                                                                
                                     batch_norm                   Y                   Y                                                                                
                                       box_clip                                       Y                                                                                
                                      box_coder                                       Y         Y                                                                      
                          collect_fpn_proposals                                       Y                                         

### 转化模型为Paddle-Lite格式
如果最后log打印出`Save the optimized model into :XXXsuccessfully`代表模型成功转换，而转换faster_rcnn_r50_fpn模型时会出现转换失败报错：
```
F0706 21:26:06.841753  1475 kernel.cc:44] Check failed: type no type registered for kernel [generate_proposals/def] output argument [RpnRoisLod]
*** Check failure stack trace: ***
Aborted (core dumped
```
因此模型部署时只使用转换后的YoloV3模型。

In [31]:
# 该手机型号是arm_v8架构，只使用CPU的话在valid_targets中只设置arm
!paddle_lite_opt \
    --model_file=inference_model/yolov3_mobilevetv3/__model__ \
    --param_file=inference_model/yolov3_mobilevetv3/__params__ \
    --optimize_out_type=naive_buffer \
    --optimize_out=mobile_cpu_model \
    --valid_targets=arm 

In [30]:
# 使用华为NPU预测时的设置
!paddle_lite_opt \
    --model_file=inference_model/yolov3_mobilevetv3/__model__ \
    --param_file=inference_model/yolov3_mobilevetv3/__params__ \
    --optimize_out_type=naive_buffer \
    --optimize_out=mobile_npu_model \
    --valid_targets=npu,arm 

In [ ]:
!paddle_lite_opt \
    --model_file=inference_model/yolov3_darknet53/__model__ \
    --param_file=inference_model/yolov3_darknet53/__params__ \
    --optimize_out_type=naive_buffer \
    --optimize_out=npu_model \
    --valid_targets=npu,arm 

In [29]:
!paddle_lite_opt \
    --model_file=inference_model/yolov3_darknet53/__model__ \
    --param_file=inference_model/yolov3_darknet53/__params__ \
    --optimize_out_type=naive_buffer \
    --optimize_out=cpu_model \
    --valid_targets=arm 

# 准备Android Studio开发环境
## 安装 Android Studio
参考[安装 Android Studio](https://developer.android.google.cn/studio/install),下载自己PC对应的环境安装。
本项目介绍在Ubuntu 18.04 64-bit上的Android Studio安装和配置。
1. 首先安装64位计算机所需的库
```shell
sudo apt-get install libc6:i386 libncurses5:i386 libstdc++6:i386 lib32z1 libbz2-1.0:i386
```
> 注意：官方教程没有提到的是，运行Android Studio需要Java环境，而且最新的OpenJDK 11会出现报错，因此需要下载OpenJDK 8
确认一下Java版本
```shell
java --version
```
下载OpenJDK 8
```shell
sudo apt-get install openjdk-8-jdk
```
如果需要，输入对应选项的数字切换Java版本
```
update-alternatives --config java
```
```shell
There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
  0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
* 2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
```

此外，部署Paddle模型还需要安装cmake和ninja环境等
```shell
# 1. Install basic software 参考官方文档，注意权限，应该是root用户
apt update
apt-get install -y --no-install-recommends \
  gcc g++ git make wget python unzip adb curl

# 2. Install cmake 3.10 or above 参考官方文档，注意权限，应该是root用户
wget -c https://mms-res.cdn.bcebos.com/cmake-3.10.3-Linux-x86_64.tar.gz && \
    tar xzf cmake-3.10.3-Linux-x86_64.tar.gz && \
    mv cmake-3.10.3-Linux-x86_64 /opt/cmake-3.10 && \  
    ln -s /opt/cmake-3.10/bin/cmake /usr/bin/cmake && \
    ln -s /opt/cmake-3.10/bin/ccmake /usr/bin/ccmake
# 3. Install ninja-build
sudo apt-get install ninja-build
```
2. 将下载的 .zip 文件解压缩到相应位置，例如 /usr/local/ 中（对于用户个人资料）或者 /opt/ 中（对于共享用户）。
3. 打开一个终端，导航至 android-studio/bin/ 目录，并执行 studio.sh，启动 Android Studio。
4. 选择是否想要导入之前的 Android Studio 设置，然后点击 OK。
5. Android Studio 设置向导将指导您完成余下的设置步骤，包括下载开发所需的 Android SDK 组件。

> 参考安装视频：
[Ubuntu上推荐的设置流程](https://developer.android.google.cn/studio/videos/studio-install-linux.mp4?hl=zh-cn)

> 注意：安装过程中会如果出现下面的报错`unable to access android sdk add-on lis`，点击`cancel`跳过。

![file](https://ss0.bdstatic.com/70cFuHSh_Q1YnxGkpoWK1HF6hhy/it/u=1251595712,1750168340&fm=26&gp=0.jpg)

## 导入Paddle-Lite-Demo
在自己的PC上下载好Paddle-Lite-Demo项目
```shell
git clone https://gitee.com/paddlepaddle/Paddle-Lite-Demo.git
```

在`PaddleLite-android-demo`目录下找到准备导入的`yolo_detection_demo`，如下图所示
![file](https://ai-studio-static-online.cdn.bcebos.com/afbaa1397f28434fa6f433f306deedb84289d1c3936c4c50b1289b1dd96c743b)

在Android Studio的欢迎界面点击`Open an existing Android Studio project`导入`yolo_detection_demo`工程
![file](https://ai-studio-static-online.cdn.bcebos.com/b87ae9237c7c4af4926dc2f57d8442137a73aeaaac6a4074a6257d034dca04a4)
## 配置SDK和NDK
进入开发界面后，点击右上角的`SDK Manager`按钮，下载好SDK和NDK
![file](https://ai-studio-static-online.cdn.bcebos.com/66e5e66c920844eaa73300ede4ea956848c84b3956d143c28a6908b45cac522e)
![file](https://ai-studio-static-online.cdn.bcebos.com/cc377161863647fc8cd22f51ad4d740731764c90b01349a5bb400897f20777e6)
![file](https://ai-studio-static-online.cdn.bcebos.com/306346149f75472a88824fb157b5b690bb7d256aca9e449a981626d6b6f5a14f)

使用快捷键`Ctrl+Alt+Shift+S`打开`Project Structure`设置好该项目的SDK和NDK路径
![file](https://ai-studio-static-online.cdn.bcebos.com/6accffab2545421caa74fa8e854a5bde95d0076332054160bbfe0f9e7be2c918)

## 修改`build.gradle`，配置国内镜像仓库
![file](https://ai-studio-static-online.cdn.bcebos.com/5a2cb44c2237443da816de6b6abe58afb3ca34bae6114ba89977f75ef04d4481)

这里其实就是将原工程`build.gradle`文件中的
```java
    repositories {
        google()
        jcenter()
        
    }
```
全部替换成对应的国内镜像加速仓库，修改后文件如下
```java
// Top-level build file where you can add configuration options common to all sub-projects/modules.

buildscript {
    repositories {
        maven { url 'https://maven.aliyun.com/repository/google/' }
        maven { url 'https://maven.aliyun.com/repository/jcenter/'}
        
    }
    dependencies {
        classpath 'com.android.tools.build:gradle:3.4.0'
        
        // NOTE: Do not place your application dependencies here; they belong
        // in the individual module build.gradle files
    }
}

allprojects {
    repositories {
        maven { url 'https://maven.aliyun.com/repository/google/' }
        maven { url 'https://maven.aliyun.com/repository/jcenter/'}
        
    }
}

task clean(type: Delete) {
    delete rootProject.buildDir
}
```
## 重新`build`项目
个人的做法是关闭当前工程再重新进入（感觉比较快），或者也可以使用快捷键`Ctrl+F9`，等待进度条跑完，最终要如同官方文档的介绍，`Build`中`Sync`要全部同步完成，看到绿色打勾才行。
![file](https://ai-studio-static-online.cdn.bcebos.com/9cc3055c40ae433486ae3aa4a870e2f300bb1bab834144f4b61a2d33c3c031c2)


# 部署模型到移动端
## 手机开发者模式连接
1. 将手机通过USB数据线连接到PC上
2. 在手机上点击：“设置”——“关于手机”——“版本号”（连续多次点击版本号才能启动开发者模式）

 <img src="https://ai-studio-static-online.cdn.bcebos.com/ae38fa53cfb44ac7a2cceee219cca9e3f78e8a47dd4b46bcbaecca48249e4a66" width = "300" height = "200" alt="yolov3_darknet53_garbage_for_npu" align=center />

3. 回到“设置”界面，可以在“搜索设置项”中查找“开发人员选项”并进入
4. **重点：先打开“仅充电模式下允许ADB调试”然后再打开“USB调试”——否则Android Studio识别不到华为手机**

 <img src="https://ai-studio-static-online.cdn.bcebos.com/551711d636ec486483c45ddea968d16f6a4c2ccb538f4b73a502b0a687a81704" width = "300" height = "200" alt="yolov3_darknet53_garbage_for_npu" align=center />

## 部署demo的CPU和NPU模型
1. 完成构建后，`PaddleLite-android-demo/yolo_detection_demo/app/src/main/assets`目录下会出现COCO数据集上的`yolov3_mobilenet_v3_for_cpu`和`yolov3_mobilenet_v3_for_hybrid_cpu_npu`两个demo模型。

2. 如果直接点击`run`，会将目标检测的CPU模型`yolov3_mobilenet_v3_for_cpu`直接部署到手机。
![file](https://ai-studio-static-online.cdn.bcebos.com/b7e28a5cd430499eae3503994911bdcc1ecb11d8c8eb4d96a518c7b5dd0cbf21)

3. 如果需要部署NPU模型，那么需要修改`PaddleLite-android-demo/yolo_detection_demo/app/src/main/res/values/strings.xml`中对应的`MODEL_DIR_DEFAULT`，将其改为`yolov3_mobilenet_v3_for_hybrid_cpu_npu`所在位置，如下图
![file](https://ai-studio-static-online.cdn.bcebos.com/8d99117dd4c144528a25796e3200a3e05a5ac2e511f349a2abe00db1a0e56b92)

4. demo中NPU模型预测效果：

 <img src="https://ai-studio-static-online.cdn.bcebos.com/7b48df24bd8a4b0cb41ff1f6dc6321f29043a98e5e5c4329ae1f12b8bb0c011d" width = "300" height = "200" alt="yolov3_darknet53_garbage_for_npu" align=center />


## 部署迁移学习模型
部署迁移学习模型其实只比前面部署**NPU上的demo模型多一个步骤**，就是修改对应label文件的路径：
1. 以`yolov3_darknet53_for_cpu`模型为例，可以在`PaddleLite-android-demo/yolo_detection_demo/app/src/main/assets/models`目录下新建一个放该模型的文件夹`yolo_v3_garbage_for_cpu`，然后将下载好的`cpu_model.nb`模型放进去并重命名为`model.nb`
2. 在`PaddleLite-android-demo/yolo_detection_demo/app/src/main/assets/labels`目录下仿照`coco-labels-2014_2017.txt`新建一个`garbage-labels.txt`内容如下
```
background
green
blue
transparent
white
yellow
red
```
3. 修改`PaddleLite-android-demo/yolo_detection_demo/app/src/main/res/values/strings.xml`中对应的`MODEL_DIR_DEFAULT`，将其改为`models/yolov3_darknet53_garbage_for_cpu`
4. 修改`PaddleLite-android-demo/yolo_detection_demo/app/src/main/res/values/strings.xml`中对应的`LABEL_PATH_DEFAULT`，将其改为`labels/garbage-labels.txt`
![file](https://ai-studio-static-online.cdn.bcebos.com/873c32bf3b3d47a6af896c4f148cdbe9e22ab42d9bd74510b25ba3661b6127d1)
## 预测效果
- `yolov3_mobilenet_v3_garbage_for_npu`:

 <img src="https://ai-studio-static-online.cdn.bcebos.com/30064752075a42f68e2c32dd962a3a51e23f697cd96647b4a03c226cf40c24e6" width = "300" height = "200" alt="yolov3_darknet53_garbage_for_npu" align=center />

- `yolov3_darknet53_garbage_for_npu`:

 <img src="https://ai-studio-static-online.cdn.bcebos.com/4245953d9aed479a8cc2ea0466eed67fd84c2390675745b5a574fc89ba83bf69" width = "300" height = "200" alt="yolov3_darknet53_garbage_for_npu" align=center />



#  小结
总结了一些使用PaddleX+Paddle-Lite部署遇到的问题，待进一步研究或者提issue
- PaddleX的deploy模块直接转化paddle-lite模型会报错
- Paddle-Lite源码编译失败
- 非armv8架构需要编译Android预测库
- 目前目标检测模型只成功部署了YoloV3，FasterRCNN会报错，DCN算子Lite尚不支持